In [1]:
import csv
import os
from collections import defaultdict
from datetime import datetime
from datetime import timedelta
from string import punctuation

import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from pandas.tslib import Timestamp

# Procesamiento de datos

## Préstamos

In [2]:
%%time

dtypes = {
    'OCUPACION': str,
    'NIVEL_EDUCACION': str,
    'AREA_ESTUDIO': str,
    'SUCURSAL_DEL_LECTOR': str,
    'COD_BARRAS_LECTOR': str,
    'CATEGORÍA': str,
}

prestamos_df = pd.read_csv('prestamos_usuarios.csv', header=0, dtype=dtypes)
prestamos_df = prestamos_df.fillna('')

CPU times: user 4.5 s, sys: 152 ms, total: 4.65 s
Wall time: 4.65 s


In [3]:
prestamos_df.shape

(3054360, 6)

In [4]:
prestamos_df.head()

,OCUPACION,NIVEL_EDUCACION,AREA_ESTUDIO,SUCURSAL_DEL_LECTOR,COD_BARRAS_LECTOR,CATEGORÍA
0,,,,ARM,88429527,CTA
1,,,,ARM,88416381,CTA
2,,,,ARM,88381353,CTA
3,empleado,técnico,adm. de empresas,ARM,88007424,CEM
4,,,,ARM,88429529,CTA


## Socios activos 2016

In [5]:
# 1.       CTA: categoría para niños y jóvenes hasta bachillerato
# 2.       CTI: categoría infantil, hasta doce años y solo es para Bogotá (blaa –biblioteca Luis Ángel Arango)
# 3.       CTB: público general (solo permite préstamo de libros)
# 4.       CTC: público en general (permite préstamo de libros y material audiovisual)
# 5.       CTF: categoría familiar, pueden acceder a los beneficios hasta tres personas del grupo familiar que se asocia
# 6.       CE1, CE2, CE2: categorías empresariales (solo para instituciones)
# 7.       CIS: categoría Inclusión Social
# 8.       CTU: categoría Universitaria
# 9.       VIP: Investigadores, préstamo de 30 materiales (libros y material audiovisual)

#      Categoría de lector: CE1 Categoría empresarial                                   218
#      Categoría de lector: CE2 Categoría empresarial 2                                   5
#      Categoría de lector: CEM Categoría Empleado/Pensionado                          1593
#      Categoría de lector: CIN Categoría Interbibliotecario                            128
#      Categoría de lector: CIP Categoría Investigador                                  351
#      Categoría de lector: CIS Categoría Inclusión Preferencial                      15974
#      Categoría de lector: CIV Categoría Investigadores                                 72
#      Categoría de lector: CML Categoría Mediador de Proyectos Culturales                2
#      Categoría de lector: CSU Categoría Sucursal                                       91
#      Categoría de lector: CTA Categoría Básica A                                    10610
#      Categoría de lector: CTB Categoría B                                            1359
#      Categoría de lector: CTC Categoría C                                             224
#      Categoría de lector: CTF Categoría familiar                                     1105
#      Categoría de lector: CTI Categoría Infantil                                     1004
#      Categoría de lector: CTU Categoría General                                     17402
#      Categoría de lector: PTE PROCESOS TÉCNICOS                                        95

In [6]:
%%time

dtypes = {
    'CODBAR': str,
    'Sucursal': str,
    'Tratamiento': str,
    'Barrio': str,
    'Ciudad': str,
    'CATEG': str,
    'Ocupación1': str,
    'Nivel Estudio1': str,
    'Área de Estudio1': str,
    'Fech. Nacim': pd.tslib.Timestamp,
    'Edad': float,
}

perfiles_df = pd.read_csv('socios_activos_2016.csv', header=0, dtype=dtypes, parse_dates=['Fech. Nacim'])
perfiles_df = perfiles_df.fillna('')

CPU times: user 460 ms, sys: 4 ms, total: 464 ms
Wall time: 464 ms


In [7]:
perfiles_df.shape

(50272, 11)

In [8]:
perfiles_df.head()

,CODBAR,Sucursal,Tratamiento,Barrio,Ciudad,CATEG,Ocupación1,Nivel Estudio1,Área de Estudio1,Fech. Nacim,Edad
0,88180782,BLAA,,chapinero,bogota,CIN,universidad,,,,
1,88183608,BLAA,sra.,ayacuyo ii,bogota,CEM,pensionado,,,1957-07-30 00:00:00,59
2,88181732,BLAA,sra.,icata,bogota,CEM,empleado,profesional,filosofía y letras,,
3,88182347,ARM,sr.,montenegro,montenegro,CEM,empleado,profesional,economía,1963-09-20 00:00:00,53
4,88181284,BLAA,sr.,la soledad,bogota,CEM,,,,,


# Grupos

In [9]:
groups_df = pd.DataFrame(perfiles_df)

#### CODBAR

In [10]:
pass

#### Sucursal

In [11]:
perfiles_df['Sucursal'].nunique()

31

In [12]:
perfiles_df['Sucursal'].value_counts()

BLAA      24278
TUN        3165
IBA        2339
PAS        2089
POP        2078
NEI        1978
PER        1922
MAN        1159
GIR        1090
IPI        1041
CAR         758
SAN         728
CASAGO      686
VAL         660
FLO         611
ARM         575
HON         544
LET         526
BUC         511
QUI         478
STM         438
SIN         438
BUE         337
BAR         328
VIL         298
CAL         287
RIO         250
CUC         236
MED         219
MON         209
             16
Name: Sucursal, dtype: int64

In [13]:
groups_df['Sucursal'] = perfiles_df['Sucursal']
groups_df = groups_df[groups_df['Sucursal'] != '']

In [14]:
groups_df.shape

(50256, 2)

In [15]:
%%time

c = perfiles_df['Sucursal'].value_counts()

def group_sucursal(x):
    e = x['Sucursal']
    if c[e] >= 1000:
        r = e
    else:
        r = 'Otro'
    return r

groups_df['Sucursal'] = groups_df.apply(group_sucursal, axis=1)

CPU times: user 1.96 s, sys: 0 ns, total: 1.96 s
Wall time: 1.96 s


#### Tratamiento

In [16]:
perfiles_df['Tratamiento'].nunique()

5

In [17]:
perfiles_df['Tratamiento'].value_counts()

sr.          22505
sra.         22341
              4995
otra inf.      392
inst            39
Name: Tratamiento, dtype: int64

In [18]:
groups_df['Tratamiento'] = perfiles_df['Tratamiento']
groups_df = groups_df[groups_df['Tratamiento'] != '']

In [19]:
groups_df.shape

(45272, 3)

In [74]:
%%time

c = perfiles_df['Tratamiento'].value_counts()

def group_tratamiento(x):
    e = x['Tratamiento']
    try:
        if e == 'sr.' or e == 'sra.':
            r = e
        else:
            r = 'Otro'
    except:
        r = 'Otro'
    return r

groups_df['Tratamiento'] = perfiles_df.apply(group_tratamiento, axis=1)

CPU times: user 1.22 s, sys: 0 ns, total: 1.22 s
Wall time: 1.22 s


#### Barrio

In [80]:
perfiles_df['Barrio'].nunique()

11332

In [81]:
perfiles_df['Barrio'].value_counts().head()

centro       1153
chapinero     429
              363
palermo       265
castilla      240
Name: Barrio, dtype: int64

In [82]:
groups_df['Barrio'] = perfiles_df['Barrio']
groups_df = groups_df[groups_df['Barrio'] != '']

In [83]:
groups_df.shape

(45014, 4)

In [84]:
%%time

c = perfiles_df['Barrio'].value_counts()

def group_barrio(x):
    e = x['Barrio']
    if c[e] >= 100:
        r = e
    else:
        r = 'Otro'
    return r

groups_df['Barrio'] = perfiles_df.apply(group_barrio, axis=1)

CPU times: user 1.94 s, sys: 0 ns, total: 1.94 s
Wall time: 1.94 s


#### Ciudad

In [92]:
perfiles_df['Ciudad'].nunique()

997

In [89]:
perfiles_df['Ciudad'].value_counts().head(40)

bogota              23087
tunja                2952
ibague               2264
popayan              1862
neiva                1754
pasto                1699
pereira              1455
ipiales               991
girardot              905
manizales             812
cartagena             721
san andres            664
valledupar            635
florencia             592
leticia               481
armenia               466
quibdo                446
santa marta           418
sincelejo             403
bucaramanga           371
pas                   365
dosquebradas          356
honda                 337
buenaventura          324
soacha                311
manizales caldas      309
villavicencio         292
cali                  269
riohacha              239
barranquilla          239
medellin              210
cucuta                207
monteria              185
pop                   145
chia                  136
                      108
floridablanca          84
bosa                   81
flandes     

In [86]:
groups_df['Ciudad'] = perfiles_df['Ciudad']
groups_df = groups_df[groups_df['Ciudad'] != '']

In [96]:
groups_df.shape

(44966, 6)

In [94]:
%%time

c = perfiles_df['Ciudad'].value_counts()

def group_ciudad(x):
    e = x['Ciudad']
    if c[e] >= 100:
        r = e
    else:
        r = 'Otro'
    return r

groups_df['Ciudad'] = perfiles_df.apply(group_ciudad, axis=1)

CPU times: user 1.89 s, sys: 0 ns, total: 1.89 s
Wall time: 1.89 s


#### CATEG

In [96]:
perfiles_df['CATEG'].nunique()

16

In [101]:
perfiles_df['CATEG'].value_counts()

CTU    17098
CIS    15759
CTA    10666
CTB     1853
CEM     1602
CTF     1121
CTI     1018
CIP      339
CTC      302
CE1      216
CIN      130
CSU       92
CIV       68
CE2        6
CML        2
           2
Name: CATEG, dtype: int64

In [97]:
groups_df['CATEG'] = perfiles_df['CATEG']
groups_df = groups_df[groups_df['CATEG'] != '']

In [98]:
groups_df.shape

(44966, 6)

In [99]:
%%time

c = perfiles_df['CATEG'].value_counts()

def group_sucursal(x):
    e = x['CATEG']
    if c[e] >= 1000:
        r = e
    else:
        r = 'Otro'
    return r

groups_df['CATEG'] = perfiles_df.apply(group_sucursal, axis=1)

CPU times: user 1.91 s, sys: 4 ms, total: 1.92 s
Wall time: 1.92 s


#### Ocupación1

In [106]:
perfiles_df['Ocupación1'].nunique()

30

In [107]:
perfiles_df['Ocupación1'].value_counts()

estudiante                 35805
empleado                    3908
docente                     2946
independiente               2549
                            1801
trabajador y estudiante     1016
pensionado                   920
hogar                        507
investigador                 258
desempleado                  181
universidad                  142
colegios                      91
cultural                      33
salud                         30
comercio                      28
instuto técnico               15
agropecuario                  12
instuto tecnológico            5
editorial                      5
alimentos                      3
financiero                     3
congregación religiosa         3
transporte                     2
construcción                   2
turismo                        2
telecomunicaciones             2
plástico                       2
minero y energético            1
bebidas                        1
automotriz                     1
Name: Ocup

In [97]:
groups_df['Ocupación1'] = perfiles_df['Ocupación1']
groups_df = groups_df[groups_df['Ocupación1'] != '']

In [98]:
groups_df.shape

(44966, 6)

In [99]:
%%time

c = perfiles_df['Ocupación1'].value_counts()

def group_ocupacion(x):
    e = x['Ocupación1']
    try:
        if e and c[e] >= 100:
            r = e
        else:
            r = 'Otro'
    except:
        r = 'Otro'
    return r

groups_df['Ocupación1'] = perfiles_df.apply(group_ocupacion, axis=1)

CPU times: user 1.91 s, sys: 4 ms, total: 1.92 s
Wall time: 1.92 s


#### Nivel Estudio1

In [50]:
perfiles_df['Sucursal'].size

50274

In [52]:
perfiles_df['Sucursal'].count()

50274

In [105]:
perfiles_df['Nivel Estudio1'].nunique()

13

In [106]:
perfiles_df['Nivel Estudio1'].value_counts()

universitario 1 nivel    9657
universitario 2 nivel    8121
profesional              6882
secundaria               6875
básica primaria          6660
                         6377
maestría                 2180
postgrado                1345
técnico                   711
graduado                  486
doctorado                 437
tecnológico               335
ninguno                   208
Name: Nivel Estudio1, dtype: int64

In [97]:
groups_df['CATEG'] = perfiles_df['CATEG']
groups_df = groups_df[groups_df['CATEG'] != '']

In [98]:
groups_df.shape

(44966, 6)

In [99]:
%%time

c = perfiles_df['CATEG'].value_counts()

def group_sucursal(x):
    e = x['CATEG']
    try:
        if c[e] >= 1000:
            r = e
        else:
            r = 'Otro'
    except:
        r = 'Otro'
    return r

groups_df['CATEG'] = perfiles_df.apply(group_sucursal, axis=1)

CPU times: user 1.91 s, sys: 4 ms, total: 1.92 s
Wall time: 1.92 s


#### Área de Estudio1

In [50]:
perfiles_df['Sucursal'].size

50274

In [52]:
perfiles_df['Sucursal'].count()

50274

In [96]:
perfiles_df['CATEG'].nunique()

16

In [98]:
perfiles_df['CATEG'].value_counts()

CTU    17098
CIS    15759
CTA    10666
CTB     1853
CEM     1602
CTF     1121
CTI     1018
CIP      339
CTC      302
CE1      216
CIN      130
CSU       92
CIV       68
CE2        6
CML        2
           2
Name: CATEG, dtype: int64

In [97]:
groups_df['CATEG'] = perfiles_df['CATEG']
groups_df = groups_df[groups_df['CATEG'] != '']

In [98]:
groups_df.shape

(44966, 6)

In [99]:
%%time

c = perfiles_df['CATEG'].value_counts()

def group_sucursal(x):
    e = x['CATEG']
    try:
        if c[e] >= 1000:
            r = e
        else:
            r = 'Otro'
    except:
        r = 'Otro'
    return r

groups_df['CATEG'] = perfiles_df.apply(group_sucursal, axis=1)

CPU times: user 1.91 s, sys: 4 ms, total: 1.92 s
Wall time: 1.92 s


#### Fech. Nacim

In [50]:
perfiles_df['Sucursal'].size

50274

In [52]:
perfiles_df['Sucursal'].count()

50274

In [96]:
perfiles_df['CATEG'].nunique()

16

In [98]:
perfiles_df['CATEG'].value_counts()

CTU    17098
CIS    15759
CTA    10666
CTB     1853
CEM     1602
CTF     1121
CTI     1018
CIP      339
CTC      302
CE1      216
CIN      130
CSU       92
CIV       68
CE2        6
CML        2
           2
Name: CATEG, dtype: int64

In [97]:
groups_df['CATEG'] = perfiles_df['CATEG']
groups_df = groups_df[groups_df['CATEG'] != '']

In [98]:
groups_df.shape

(44966, 6)

In [99]:
%%time

c = perfiles_df['CATEG'].value_counts()

def group_sucursal(x):
    e = x['CATEG']
    try:
        if c[e] >= 1000:
            r = e
        else:
            r = 'Otro'
    except:
        r = 'Otro'
    return r

groups_df['CATEG'] = perfiles_df.apply(group_sucursal, axis=1)

CPU times: user 1.91 s, sys: 4 ms, total: 1.92 s
Wall time: 1.92 s


#### Edad

In [50]:
perfiles_df['Sucursal'].size

50274

In [52]:
perfiles_df['Sucursal'].count()

50274

In [96]:
perfiles_df['CATEG'].nunique()

16

In [98]:
perfiles_df['CATEG'].value_counts()

CTU    17098
CIS    15759
CTA    10666
CTB     1853
CEM     1602
CTF     1121
CTI     1018
CIP      339
CTC      302
CE1      216
CIN      130
CSU       92
CIV       68
CE2        6
CML        2
           2
Name: CATEG, dtype: int64

In [97]:
groups_df['CATEG'] = perfiles_df['CATEG']
groups_df = groups_df[groups_df['CATEG'] != '']

In [98]:
groups_df.shape

(44966, 6)

In [99]:
%%time

c = perfiles_df['CATEG'].value_counts()

def group_sucursal(x):
    e = x['CATEG']
    try:
        if c[e] >= 1000:
            r = e
        else:
            r = 'Otro'
    except:
        r = 'Otro'
    return r

groups_df['CATEG'] = perfiles_df.apply(group_sucursal, axis=1)

CPU times: user 1.91 s, sys: 4 ms, total: 1.92 s
Wall time: 1.92 s


In [100]:
groups_df.head()

,CODBAR,Sucursal,Tratamiento,Ciudad,CATEG
1,88183608,BLAA,sra.,bogota,CEM
2,88181732,BLAA,sra.,bogota,CEM
3,88182347,Otra,sr.,Otro,CEM
4,88181284,BLAA,sr.,bogota,CEM
6,88183786,BLAA,sr.,bogota,CEM


In [50]:
for e, c in perfiles_df['Sucursal'].value_counts().iteritems():
    if c >= 1000:
        groups_df[]

('BLAA  ', 24278)


#### Tratamiento

In [29]:
perfiles_df['Tratamiento'].nunique()

5

In [30]:
perfiles_df['Tratamiento'].value_counts()

sr.          22505
sra.         22341
              4997
otra inf.      392
inst            39
Name: Tratamiento, dtype: int64

In [ ]:
%%time

def bins_to_labels(bins):
    labels = []
    labels.append('{}-'.format(bins[1]))
    for i in range(1, len(bins) - 2):
        labels.append('{}-{}'.format(bins[i]+1, bins[i+1]))
    labels.append('{}+'.format(bins[-2]))
    return labels

bins = [-1, 11, 17, 25, 65, 1000]
bins = [-1, 15,28,40,50,65, 1000]
labels = bins_to_labels(bins)

def calculate_rango(x):
    pd.cut(pd.Series(range(101)), bins=bins, labels=labels)

perfiles_df['Rango'] = pd.cut(perfiles_df['Edad'], bins=bins, labels=labels)

In [ ]:
perfiles_df['Rango'].value_counts()

In [ ]:
perfiles_df['Edad'].value_counts()